In [1]:
import numpy
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Dense, Activation, Dropout, Lambda, Multiply, Add, Concatenate, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [2]:
def sub_module(inp_shape=(4096, 1)):
    
    # convolutional operation parameters
    n_filters = 32 
    filter_width = 2
    dilation_rates = [2**i for i in range(11)] * 3 

    # define an input history series and pass it through a stack of dilated causal convolution blocks. 
    Input_seq = Input(shape=inp_shape, dtype='float32')
    x = Input_seq

    skips = []
    for dilation_rate in dilation_rates:

        # preprocessing - equivalent to time-distributed dense
        x = Conv1D(16, 1, padding='same', activation='relu')(x) 

        # filter convolution
        x_f = Conv1D(filters=n_filters,
                     kernel_size=filter_width, 
                     padding='same',
                     dilation_rate=dilation_rate)(x)

        # gating convolution
        x_g = Conv1D(filters=n_filters,
                     kernel_size=filter_width, 
                     padding='same',
                     dilation_rate=dilation_rate)(x)

        # multiply filter and gating branches
        z = Multiply()([Activation('tanh')(x_f),
                        Activation('sigmoid')(x_g)])

        # postprocessing - equivalent to time-distributed dense
        z = Conv1D(16, 1, padding='same', activation='relu')(z)

        # residual connection
        x = Add()([x, z])    
sub_mod_A
        # collect skip connections
        skips.append(z)

    # add all skip connection outputs 
    out = Activation('relu')(Add()(skips))
    
    sub_module = Model(Input_seq, out)
    
    return sub_module


def full_module(inp_shape=(4096, 2)):
    
    # define an input history series and pass it through a stack of dilated causal convolution blocks. 
    Input_seq = Input(shape=inp_shape, dtype='float32')
    
    x_A = Lambda(lambda y: y[:,:,0])(Input_seq); x_A = Reshape(target_shape=(4096,1))(x_A)
    x_B = Lambda(lambda y: y[:,:,1])(Input_seq); x_B = Reshape(target_shape=(4096,1))(x_B)
    
    sub_mod_A = sub_module(inp_shape=(4096, 1))
    sub_mod_B = sub_module(inp_shape=(4096, 1))
    
    x_A = sub_mod_A(x_A)
    x_B = sub_mod_B(x_B)
    
    out = tf.keras.layers.Concatenate(axis=-1)([x_A, x_B])
    'sigmoid'
    # final time-distributed dense layers 
    out = Conv1D(128, 1, padding='same')(out)
    out = Activation('relu')(out)
    #out = Dropout(.2)(out)
    out = Conv1D(1, 1, padding='same')(out)
    
    #out = Reshape((8192,))(out)
    out = Activation('sigmoid')(out)
    
    model = Model(Input_seq, out)
    
    return model

In [3]:
M = full_module()

In [4]:
M.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4096, 2)]    0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 4096)         0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 4096, 1)      0           lambda[0][0]                     
_______________________________________________________________________________________

In [8]:
# import tensorflow as tf
# import hls4ml
# #import plotting
# import matplotlib.pyplot as plt
# import yaml
# config = hls4ml.utils.config_from_keras_model(M, granularity='name')
# config['Model']['Precision'] = 'ap_fixed<16,6>'
# config['Model']['ReuseFactor'] = 1
# for i in config['LayerName']:
#     config['LayerName'][i]['ReuseFactor'] = 100
# config['Model']['Strategy'] = 'Resource'
# #config['LayerName']['soft']['Precision'] = 'ap_fixed<20,8>'
# #config['LayerName']['dense1']['Strategy'] = 'Resource'
# config['LayerName']['soft']['Strategy'] = 'Stable'
#plotting.print_dict(config)
# hls_model = hls4ml.converters.convert_from_keras_model(model,
#                                                        output_dir='resnet50/hls4ml_prj',
#                                                        fpga_part='xcu250-figd2104-2L-e',
#                                                        io_type = 'io_stream',
#                                                        hls_config = config)

Interpreting Model
Topology:
Layer name: input_1, layer type: Input


Exception: ERROR: Unsupported layer type: Lambda